This notebook computes all of the "distances" (the fuzzy logic scores for IOU, DICE, ...) between the GradCam maps produced by the final resNet50 model and the visual characterstics maps created by the derm. That is: 

```
for each image i  
  for each GradCam image i_gc  
    for each derm d  
      for each characteristic d_char  
        compute the visual distance between i_gc and d_char 
```


In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
from pathlib import Path
from collections import defaultdict
from matplotlib.pyplot import imshow
from PIL import Image


In [2]:
# Set paths

# Path to the GradCam images.
gradcam_main_path = Path('/home/ubuntu/store/efficientnet-final-size/visualisation/gradcam/')

# Set path to the model used to create the visualisations.
model_path = Path('efficientnetb4_0')

# Path to the masks create by the derms for the individual characteristics.
derm_mask_main_path = Path('/home/ubuntu/store/masks/masks_resized')

# Path to the rescaled test images.
test_im_path = Path('/home/ubuntu/store/DermX-test-set/test/')


In [3]:
full_gradcam_path = gradcam_main_path / model_path
full_lime_path = lime_main_path / model_path

In [4]:
classes = [
    "Acne",
    "Actinic keratosis",
    "Psoriasis",
    "Seborrheic dermatitis",
    "Viral warts",
    "Vitiligo"
]

derms = [
    'derm0',
    'derm1',
    'derm2',
    'derm3',
    'derm4',
    'derm5',
    'derm6',
    'derm7',
]

chars = [
    'closed-comedo',
    'cyst',
    'dermatoglyph-disruption',
    'leukotrichia',
    'macule',
    'nodule',
    'open-comedo',
    'papule',
    'patch',
    'plaque',
    'pustule',
    'scale',
    'scar',
    'sun-damage',
    'telangiectasia',
    'thrombosed-capillaries'
]


In [5]:
# Extract the paths to the derm masks. The resulting list is only used for a sanity check.
derm_mask_paths = [p for p in derm_mask_main_path.iterdir() if p.suffix == '.png']
derm_mask_paths[0:5]

[PosixPath('/home/ubuntu/store/masks/masks_spinaltap/043023HB_mihaela_2021-05-27-masks_plaque.png'),
 PosixPath('/home/ubuntu/store/masks/masks_spinaltap/043212HB_adelina_2021-05-27-masks_scale.png'),
 PosixPath('/home/ubuntu/store/masks/masks_spinaltap/043269HB_adelina_2021-05-27-masks_papule.png'),
 PosixPath('/home/ubuntu/store/masks/masks_spinaltap/293--WatermarkedWyJXYXRlcm1hcmtlZCJd_mihaela_2021-05-27-masks_scale.png'),
 PosixPath('/home/ubuntu/store/masks/masks_spinaltap/viral-wart-08--WatermarkedWyJXYXRlcm1hcmtlZCJd_oana_2021-05-27-masks_papule.png')]

In [6]:
# Extract the paths to the test images. Only needed for visualization and debugging.
test_img_paths = [p for p in Path(test_im_path).rglob('*.jpeg')]
test_img_paths[0:5]

[PosixPath('/home/ubuntu/store/DermX-test-set/test/Actinic keratosis/017359HB.jpeg'),
 PosixPath('/home/ubuntu/store/DermX-test-set/test/Actinic keratosis/476--WatermarkedWyJXYXRlcm1hcmtlZCJd.jpeg'),
 PosixPath('/home/ubuntu/store/DermX-test-set/test/Actinic keratosis/469--WatermarkedWyJXYXRlcm1hcmtlZCJd.jpeg'),
 PosixPath('/home/ubuntu/store/DermX-test-set/test/Actinic keratosis/3742--WatermarkedWyJXYXRlcm1hcmtlZCJd.jpeg'),
 PosixPath('/home/ubuntu/store/DermX-test-set/test/Actinic keratosis/3753--WatermarkedWyJXYXRlcm1hcmtlZCJd.jpeg')]

In [7]:
# Metrics supporting probabilistic segmentation maps
fuzzy_and = lambda x,y: np.minimum(x,y)
fuzzy_or = lambda x,y: np.maximum(x,y)
fuzzy_not = lambda x: 1-x

def pixel_metrics_fuzzy(y_true, y_pred):
    """
    Pixel-level metrics of segmentation accuracy following fuzzy logic operators.
    
    :param y_true: numpy.ndarray of reference segmentation, values in [0,1]
    :param y_pred: numpy.ndarray of predicted segmentation, values in [0,1]

    :return: a dictionary encoding the metrics
    """
        
    np.testing.assert_equal(y_true.shape, y_pred.shape, err_msg="Expecting \
    the reference and predicted segmentations to be of the same size.")
    
    # Check the ranges
    np.testing.assert_equal(np.logical_and(y_true >= 0, y_true <= 1).all(), True, err_msg="Expecting \
    the reference segmentations to be in the range 0 to 1.")
    np.testing.assert_equal(np.logical_and(y_pred >= 0, y_pred <= 1).all(), True, err_msg="Expecting \
    the predicted segmentations to be in the range 0 to 1.")
    
    TP = fuzzy_and(y_true, y_pred).sum()
    TN = fuzzy_and(fuzzy_not(y_true), fuzzy_not(y_pred)).sum()
    union = fuzzy_or(y_true, y_pred).sum()
    
    metrics = {}
    
    # Summary metrics
    metrics["iou"] = TP / union
    metrics["dice"] = 2 * TP / ( y_true.sum() + y_pred.sum() ) 
    
    # Positive class metrics
    metrics["precision"] = TP / y_pred.sum()
    metrics["recall"] = TP / y_true.sum()
    
    # Negative class metrics
    metrics["negative_predictive_value"] = TN / fuzzy_not(y_pred).sum()
    metrics["specificity"] = TN / fuzzy_not(y_true).sum()
    
    return metrics

In [8]:
def calc_res(gradcam_image_path, derm_char_mask_path, interpolation_method=Image.NEAREST):
    """
    Calculates the fuzzy logic metrics given the paths to a pair of input images.
    The derm mask is resized to match the size of the gradcam image.
    
    Input:
    - gradcam_image_path: Pathlib path to a gradCam image. The file is assumed to be in .npy format.
    - derm_char_mask_path: Pathlib path to a derm annotation. The file is assumed to be in a format that can
                           be opened by PIL.
    - interpolation_method: String. The method used for interpolation when resizing the derm mask. Options are
                            NEAREST, BOX, BILINEAR, HAMMING, BICUBIC, LANCZOS. Default is NEAREST.    
    """
    
    # Open images.
    gradcam_im = np.load(gradcam_image_path, allow_pickle=True)
    mask_im = Image.open(derm_char_mask_path)
    
    # Resize the derm mask if its size does not match the size of the gradcam image.
    if gradcam_im.shape != mask_im.size[::-1]:
        # Note that resize uses (cols, rows) format, while .shape is in (rows, cols) format.
        mask_im = mask_im.resize((gradcam_im.shape[::-1]), interpolation_method)
    
    # Corvert the derm mask to numpy format and normalize to [0, 1].
    mask_im = np.asarray(mask_im) / 255
  
    res = pixel_metrics_fuzzy(mask_im, gradcam_im)
    return res

In [9]:
# For each image in the test set, calculate the value of the defined metrics given the GradCam image for a given
# class and the outline made by a specific derm for a given characteristic. The result is stored in a defaultdict.
# As a sanity check, the number of matches between GradCam images and derm annotations is also calculated.
rec_dd = lambda: defaultdict(rec_dd)
out_gradcam = rec_dd()
hit_counter = 0

# Set the method used for interpolation when resizing the derm annotations.
interpolation = Image.NEAREST

for p in test_img_paths:
    for c in classes:
        # Check if matching GradCam file exists.
        gc_path = full_gradcam_path / Path(p.stem + '_' + c + '.npy')
        if gc_path.is_file():
            for d in derms:
                for ch in chars:
                    # Check if this derm has created a mask for this characteristic.
                    mask_path = derm_mask_main_path / Path(p.stem + '_' + d + '_2021-05-27-masks_' + ch + '.png') 
                    if mask_path.is_file():
                        # Calculate the value of the metics given the GradCam image and the derm mask.
                        gradcam_metric_val = calc_res(gc_path, mask_path, interpolation)
                        out_gradcam[p.stem][c][d][ch] = gradcam_metric_val
                        hit_counter += 1
        else:
            print('GradCam or LIME file missing for image: ', p)

print(hit_counter)



19578


In [12]:
# We expect that the value of the hit counter should be equal to the number of derm masks multiplied with the number
# of classes.
if len(derm_mask_paths)*6 != hit_counter:
    print('Oh no, some files were not found. Expected/found: ', len(derm_mask_paths)*6, hit_counter)
else:
    print('GradCam images found for all derm annotations.')


GradCam images found for all derm annotations.


In [13]:
def transform_to_list(nested_dict):
    # Transform the gradcam defaultdict to a list of tuples.
    
    out = []
    for im_name, class_dict in nested_dict.items():
        for class_name, derm_dict in class_dict.items():
            for derm_name, char_dict in derm_dict.items():
                for char_name, metric_dict in char_dict.items():
                    tmp = tuple(metric_dict.values())
                    out.append( (im_name, class_name, derm_name, char_name) + tmp )
    return out

In [14]:
gradcam_res_list = transform_to_list(out_gradcam)
len(gradcam_res_list)

19578

## Make output dataFrames/csv files

In [18]:
col_names = ['image_name',
             'visualisation_class',
             'derm',
             'characteristic',
             'iou',
             'dice',
             'precision',
             'recall',
             'negative_predictive_value',
             'specificity'
            ]

In [ ]:
gradcam_df = pd.DataFrame.from_records(gradcam_res_list, columns=col_names)
gradcam_df

In [22]:
# Save dataFrames
model_name = str(model_path)
gradcam_df.to_csv(model_name + "_gradcam_scores.csv")


# Filter

In [123]:
import glob
model_names = glob.glob('/home/ubuntu/store/efficientnet-final-size/*h5')

In [124]:
model_names

['/home/ubuntu/store/efficientnet-final-size/efficientnetb4_4.h5',
 '/home/ubuntu/store/efficientnet-final-size/efficientnetb4_0.h5',
 '/home/ubuntu/store/efficientnet-final-size/efficientnetb4_2.h5',
 '/home/ubuntu/store/efficientnet-final-size/efficientnetb4_3.h5',
 '/home/ubuntu/store/efficientnet-final-size/efficientnetb4_1.h5']

In [125]:
model_name = Path(model_names[1]).stem

In [126]:
def get_table(results_pred):
    results_mean = results_pred.groupby('characteristic').median()
    results_std =  results_pred.groupby('characteristic').mad()
    
    ### Add means
    results_mean.loc['mean'] = results_mean.mean()
    results_std.loc['mean']   = results_mean.mean()
    
    columns = results_mean.columns.to_list()

    table_pred = results_mean
    return table_pred

class_map = {'0' : 'Acne' ,
             '1' : 'Actinic keratosis',
             '2' : 'Psoriasis' ,
             '3' : 'Seborrheic dermatitis',
             '4' : 'Viral warts',
             '5' : 'Vitiligo'}

filefolder = model_name + "_gradcam_scores.csv"
predsfile = '/home/ubuntu/store/efficientnet-final-size/' + model_name + '_preds.csv'

In [127]:
filtered_subjects = pd.read_csv('./include_images_525.csv')
filtered_subjects['image_id'] = filtered_subjects['image_id'].apply(lambda x: Path(x).stem)
filtered_subjects = filtered_subjects.rename(columns={'image_id': 'image_name'})
filtered_subjects = filtered_subjects.drop(columns=['Unnamed: 0'])

In [128]:
df_preds = pd.read_pickle(predsfile)
gradcam_df = pd.read_csv(filefolder)
gradcam_df=gradcam_df.drop(['Unnamed: 0'], axis=1)

In [129]:
df_preds['pred'] =  df_preds['pred'].apply(lambda x: class_map[str(x)])
df_preds['actual'] = df_preds['actual'].apply(lambda x: class_map[str(x)])
df_preds['filenames'] = df_preds['filenames'].apply(lambda x: Path(x).stem)
df_preds = df_preds.rename(columns={'filenames':'image_name'})
df_preds = df_preds.merge(filtered_subjects, on = 'image_name')

In [32]:
result = pd.merge(gradcam_df,df_preds, on = 'image_name')

results_pred = result[result.visualisation_class == result.pred]
table_pred = get_table(results_pred)
# Get it for actual
results_actual = result[result.visualisation_class == result.actual  ]
table_actual = get_table(results_actual)

table_pred.to_pickle('./' + model_name + '_gradcam_visualisation_scores_pred.pkl')
table_actual.to_pickle('./' + model_name + '_gradcam_visualisation_scores_actual.pkl')

results_equal = result[ (result.actual == result.pred) & (result.visualisation_class == result.pred) ]
table_equal = get_table(results_equal)

results_diff = result[ (result.actual != result.pred) & (result.visualisation_class == result.pred) ]
table_diff = get_table(results_diff)

table_equal.to_pickle('./' + model_name + '_gradcam_visualisation_scores_equal.pkl')
table_diff.to_pickle('./' + model_name + '_gradcam_visualisation_scores_diff.pkl')

# Creating Benchmarks

## By concatenation of all models

In [89]:
def lt_format(results_performance, col_n):
    results_mean = results_performance.groupby(col_n).mean()
    results_std = results_performance.groupby(col_n).std()
    
    results_mean.loc['mean'] = results_mean.mean()
    results_std.loc['mean'] = results_std.mean()
    for col in results_performance.columns.to_list():
        results_mean[col] = results_mean[col].apply(lambda x: f'{np.round(x,decimals=2)} $\pm$ ')
        results_std[col] = results_std[col].apply(lambda x: f'{np.round(x,decimals=2)}')
    
    return results_mean + results_std

model_names = glob.glob('/home/ubuntu/store/efficientnet-final-size/*h5')
results_actual = pd.DataFrame()
results_pred = pd.DataFrame()
results_equal = pd.DataFrame()
results_diff = pd.DataFrame()
results_performance = pd.DataFrame()

for mn in sorted(model_names):
    model_name = Path(mn).stem
    performance = pd.read_pickle('./' + model_name + '_performance.pkl')
    actual = pd.read_pickle('./' + model_name + '_gradcam_visualisation_scores_actual.pkl').loc[:,['recall','specificity','dice']]
    pred = pd.read_pickle('./' + model_name + '_gradcam_visualisation_scores_pred.pkl').loc[:,['recall','specificity','dice']]
    equal = pd.read_pickle('./' + model_name + '_gradcam_visualisation_scores_equal.pkl').loc[:,['recall','specificity','dice']]
    diff = pd.read_pickle('./' + model_name + '_gradcam_visualisation_scores_diff.pkl').loc[:,['recall','specificity','dice']]
    
    performance=performance.drop('mean')
    actual = actual.drop('mean')
    pred = pred.drop('mean')
    equal = equal.drop('mean')
    diff = diff.drop('mean')
    
    results_performance = pd.concat([results_performance, performance])
    results_actual = pd.concat([results_actual, actual])
    results_pred = pd.concat([results_pred, pred]) 
    results_diff = pd.concat([results_diff, diff])

results_performance.index.name = 'label'

table_performance = lt_format(results_performance, 'label')
table_actual = lt_format(results_actual, 'characteristic')

In [90]:
table_performance = table_performance.drop(columns = ['accuracy'])

In [91]:
table_performance

,precision,f1-score,specificity
label,,,
acne,0.77 $\pm$ 0.16,0.65 $\pm$ 0.32,0.97 $\pm$ 0.02
actinic_keratosis,0.74 $\pm$ 0.08,0.55 $\pm$ 0.11,0.96 $\pm$ 0.02
psoriasis_no_pustular,0.43 $\pm$ 0.14,0.57 $\pm$ 0.12,0.67 $\pm$ 0.23
seborrheic_dermatitis,0.62 $\pm$ 0.08,0.45 $\pm$ 0.22,0.95 $\pm$ 0.03
vitiligo,0.0 $\pm$ 0.01,0.0 $\pm$ 0.01,0.9 $\pm$ 0.03
wart,0.08 $\pm$ 0.04,0.07 $\pm$ 0.04,0.86 $\pm$ 0.04
mean,0.44 $\pm$ 0.08,0.38 $\pm$ 0.14,0.89 $\pm$ 0.06


In [87]:
print(table_performance.to_latex())

\begin{tabular}{llll}
\toprule
{} &        precision &         f1-score &      specificity \\
label                 &                  &                  &                  \\
\midrule
acne                  &  0.77 \$\textbackslash pm\$ 0.16 &  0.65 \$\textbackslash pm\$ 0.32 &  0.97 \$\textbackslash pm\$ 0.02 \\
actinic\_keratosis     &  0.74 \$\textbackslash pm\$ 0.08 &  0.55 \$\textbackslash pm\$ 0.11 &  0.96 \$\textbackslash pm\$ 0.02 \\
psoriasis\_no\_pustular &  0.43 \$\textbackslash pm\$ 0.14 &  0.57 \$\textbackslash pm\$ 0.12 &  0.67 \$\textbackslash pm\$ 0.23 \\
seborrheic\_dermatitis &  0.62 \$\textbackslash pm\$ 0.08 &  0.45 \$\textbackslash pm\$ 0.22 &  0.95 \$\textbackslash pm\$ 0.03 \\
vitiligo              &   0.0 \$\textbackslash pm\$ 0.01 &   0.0 \$\textbackslash pm\$ 0.01 &   0.9 \$\textbackslash pm\$ 0.03 \\
wart                  &  0.08 \$\textbackslash pm\$ 0.04 &  0.07 \$\textbackslash pm\$ 0.04 &  0.86 \$\textbackslash pm\$ 0.04 \\
mean                  &  0.44 \

In [39]:
table_actual

,recall,specificity,dice
characteristic,,,
closed-comedo,0.21 $\pm$ 0.1,0.83 $\pm$ 0.04,0.08 $\pm$ 0.03
cyst,0.2 $\pm$ 0.14,0.85 $\pm$ 0.03,0.02 $\pm$ 0.01
dermatoglyph-disruption,0.09 $\pm$ 0.05,0.79 $\pm$ 0.06,0.05 $\pm$ 0.02
leukotrichia,0.14 $\pm$ 0.1,0.82 $\pm$ 0.04,0.07 $\pm$ 0.03
macule,0.27 $\pm$ 0.07,0.8 $\pm$ 0.03,0.09 $\pm$ 0.03
nodule,0.2 $\pm$ 0.08,0.82 $\pm$ 0.03,0.03 $\pm$ 0.01
open-comedo,0.19 $\pm$ 0.1,0.83 $\pm$ 0.04,0.08 $\pm$ 0.04
papule,0.23 $\pm$ 0.07,0.8 $\pm$ 0.03,0.07 $\pm$ 0.01
patch,0.28 $\pm$ 0.06,0.8 $\pm$ 0.03,0.21 $\pm$ 0.04


In [40]:
print(table_actual[['dice','recall','specificity']].to_latex())

\begin{tabular}{llll}
\toprule
{} &             dice &           recall &      specificity \\
characteristic          &                  &                  &                  \\
\midrule
closed-comedo           &  0.08 \$\textbackslash pm\$ 0.03 &   0.21 \$\textbackslash pm\$ 0.1 &  0.83 \$\textbackslash pm\$ 0.04 \\
cyst                    &  0.02 \$\textbackslash pm\$ 0.01 &   0.2 \$\textbackslash pm\$ 0.14 &  0.85 \$\textbackslash pm\$ 0.03 \\
dermatoglyph-disruption &  0.05 \$\textbackslash pm\$ 0.02 &  0.09 \$\textbackslash pm\$ 0.05 &  0.79 \$\textbackslash pm\$ 0.06 \\
leukotrichia            &  0.07 \$\textbackslash pm\$ 0.03 &   0.14 \$\textbackslash pm\$ 0.1 &  0.82 \$\textbackslash pm\$ 0.04 \\
macule                  &  0.09 \$\textbackslash pm\$ 0.03 &  0.27 \$\textbackslash pm\$ 0.07 &   0.8 \$\textbackslash pm\$ 0.03 \\
nodule                  &  0.03 \$\textbackslash pm\$ 0.01 &   0.2 \$\textbackslash pm\$ 0.08 &  0.82 \$\textbackslash pm\$ 0.03 \\
open-comedo          